<br>
<img src="https://sites.google.com/a/chesterqualls.com/chester-qualls-is-it/ChesterQualls%20Logo%28small%29.png?revision=2">
<br>
***
# A Mind - Enroll Data

> My goal is to see if I can create easily understood and reusable code to mungle data.

> This notebook deals with the Enroll data and follows somewhat the path seen in the R script done by JW. This notebook takes a slightly different approach than that taken by JW.
> * This notebook only deals with the Enrollment Data.
* It also doesn't strip out as much detail as JW did.  This keeps more information so that we have more options later.  
* Getting information into the form that JW was shooting for is to be shaped later specifically for the analysis or test to be done.  

### * We begin with the Enroll File

In [2]:
import pandas as pd
import numpy as np
import os

#This next line supresses warning message, which may be nice before exporting to send
pd.options.mode.chained_assignment = None
#pd.set_option('chained_assignment','warn')

os.chdir("c:/Users/Chester Qualls/Desktop/Customers/EDU_Example")

print (os.getcwd())

c:\Users\Chester Qualls\Desktop\Customers\EDU_Example


In [4]:
#Import and combine enroll files
df_enroll_cohort = pd.read_csv('CRI Student Data/A Mind_Cohort 1_Enrollment.csv')

df_enroll_comp = pd.read_csv('CRI Student Data/A Mind_Comp 1_Enrollment.csv')

#frames = (df_enroll_cohort, df_enroll_comp)

df_enroll_all = pd.concat([df_enroll_cohort,df_enroll_comp]) # or df_enroll_all = pd.concat(frames)

df_enroll_all.columns = df_enroll_all.columns.str.lower()

#The following schools dropped out of the study
df_enroll_all = df_enroll_all.loc[df_enroll_all['schoolcode'] != 2876]
df_enroll_all = df_enroll_all.loc[df_enroll_all['schoolcode'] != 3648]
df_enroll_all = df_enroll_all.loc[df_enroll_all['schoolcode'] != 2134]
df_enroll_all = df_enroll_all.loc[df_enroll_all['schoolcode'] != 5324]

#Example for changing code because ix is deprecated
#df_grades_ia = df_grades_all.loc[df_grades_all['is_ia'] == 1]

#Only need the following columns from the Enroll data
df_enroll_all = df_enroll_all[['researchid','districtcode','schoolcode', 'schoolname','ayd_ia_c1_comp','ayd__ia_cohort_1',
                               'schoolyear', 'gradelevel','gender','disabilityflag','isstudentimmigrant','primarylanguage', 
                               'federalethracerollupcode','federalethracerollupname','expectedgradyear','cumulativegpa',
                               'cumulativecreditsattempted','cumulativecreditsearned','numdayspresent']]

#Change some of the column names for clarity
df_enroll_all = df_enroll_all.rename(columns={'disabilityflag': 'is_sped', 'isstudentimmigrant': 'is_immigrant', 
                                              'primarylanguage':'is_ell','ayd_ia_c1_comp':'comparison',
                                              'ayd__ia_cohort_1':'cohort'})

#print df_enroll_all.info
print ("Finished")

C:\Users\Chester Qualls\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18,36,37,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Chester Qualls\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (18,36,37,39,40,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Finished


## Change the content of cells (Enroll)

> In this section I will change a few columns from a character to a binary boolean.  This is apparently more helpful in analyzing those columns.

In [5]:
df_enroll_all['is_sped'] = df_enroll_all['is_sped'].map({'Y': 1, 'N': 0})
df_enroll_all['gender'] = df_enroll_all['gender'].map({'M': 1, 'F': 0})
df_enroll_all['is_immigrant'] = df_enroll_all['is_immigrant'].map({'Y': 1, 'N': 0})

#If Map deprecated maybe could use something like this
#w.female.replace(to_replace=dict(female=1, male=0), inplace=True)


df_enroll_all['cohort'] = np.where(df_enroll_all['cohort']=='y',1,0)
df_enroll_all['comparison'] = np.where(df_enroll_all['comparison']=='y',1,0)
df_enroll_all['is_ell'] = np.where(df_enroll_all['is_ell']=='English',0,1)
#df_enroll_all['gradelevel'] = np.where(df_enroll_temp['gradelevel']=='9','09',df_enroll_temp['gradelevel'])
#The above code is commented out because it didn't work.  I don't believe there was an error, but since it's a numeric
#field it still ended up as 9.  Don't know if we would need to convert to character field??

#print df_enroll_all.info
print ("Finished")

Finished


## Eliminate Duplicates (Enroll)
> * Time to reduce the number of rows.
* Determine if we want to keep past years in this table
* Also remove any records that represent attendance of less than 90 days


In [6]:
# This groups the data by the named fields and then drops the duplicates keeping the last record
# Keeping only the last may not be the best solution, may want to avg or sum a column first. Dropped 387 rows.

df_enroll_all = df_enroll_all.drop_duplicates(['researchid', 'schoolcode', 'schoolyear', 'gradelevel'], keep ='last')

# Remove rows with less than 90 days of attendance. Dropped 14682 rows.
df_enroll_all = df_enroll_all.loc[(df_enroll_all['numdayspresent'] > 90)]

print ("Finished")

Finished


## Add new rows, and fill them (Enroll)
> This section will determine if a school is AYD or IA.

In [7]:
df_ayd_schools = pd.DataFrame({'schoolcode':[3201, 3109, 4113, 5286, 2706, 2264],'is_ayd':[1,1,1,1,1,1]}, 
                              columns=['schoolcode','is_ayd'])
df_ia_schools = pd.DataFrame({'schoolcode':[2553, 5171, 2876, 3314, 2580, 3576, 4515, 4196, 4254, 3468, 3141, 
                              3648, 2623, 2478, 2343, 2706, 2134, 3311], 'is_ia':[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]}, 
                             columns=['schoolcode', 'is_ia'])

df_enroll_all = pd.merge(df_enroll_all, df_ayd_schools, how='left', on='schoolcode')
df_enroll_all = pd.merge(df_enroll_all, df_ia_schools, how='left', on='schoolcode')

df_enroll_all['is_ayd'] = np.where(df_enroll_all['is_ayd'] == 1,1,0)
df_enroll_all['is_ia'] = np.where(df_enroll_all['is_ia'] == 1,1,0)

print (df_enroll_all.info)



<bound method DataFrame.info of        researchid  districtcode  schoolcode              schoolname  \
0          521222         37501        2553      Salame High School   
1          425041         37501        2553      Salame High School   
2          522530         37501        2553      Salame High School   
3          591302         37501        2553      Salame High School   
4          831690         37501        2553      Salame High School   
5         1091694         37501        2553      Salame High School   
6         1284910         37501        2553      Salame High School   
7         1157271         37501        2553      Salame High School   
8         1091258         37501        2553      Salame High School   
9           42000         37501        2553      Salame High School   
10         652883         37501        2553      Salame High School   
11         930356         37501        2553      Salame High School   
12         806566         37501        2553  

In [8]:
#first get a list of the researchid's we have in the enroll table, can't do much if we don't have them
#this will be reused often to limit other files to only available enrollment files

df_researchid_all = df_enroll_all [['researchid']]
df_researchid_all = df_researchid_all.drop_duplicates(['researchid'], keep ='first')

df_researchid_all.to_pickle('pickles/am_researchid_all.pickle')

print ("Finished")

Finished


## Export CSV and create Pickle

> The files created here are to be used by tests or analysis done later.  This could be done in Python or in R.  
> * Analysis to be done is SPSS may require a specific combination of files, or limiting of records that will answer the specific questions being done.

In [9]:
df_enroll_all.to_csv("clean_data/am_enroll_clean.csv")
df_enroll_all.to_pickle('pickles/am_enroll_clean.pickle')


print ("Finished")

Finished
